In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np           
import numpy
from numpy import array
import matplotlib.pyplot as plt
import math
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [3]:
#code for MAPE, referred from the url: https://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-percentage-error
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
x=pd.read_csv('/content/drive/My Drive/fyp/Lagard-less till 2021.csv',parse_dates=['Date'],index_col='Date')
x = x.loc["2016-01-01" :]                         #Since 2016-01-01, 5y(1238rows till 2020-12-31), + year 2021's rows
y=x.copy()                                        #deep copy
x.reset_index(drop=True, inplace=True)
x

,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BPCL,BHARTIARTL,BRITANNIA,CIPLA,DIVISLAB,DRREDDY,GAIL,GRASIM,HCLTECH,HDFCBANK,HEROMOTOCO,HINDALCO,HINDUNILVR,HDFC,ICICIBANK,ITC,IOC,INDUSINDBK,INFY,JSWSTEEL,KOTAKBANK,LT,M&M,NTPC,ONGC,POWERGRID,RELIANCE,SBIN,SUNPHARMA,TCS,TATAMOTORS,TATASTEEL,TECHM,TITAN,UPL,ULTRACEMCO,WIPRO
0,267.55,878.75,449.90,2518.00,6095.85,896.05,340.50,2986.40,655.35,1162.50,3108.60,369.75,3767.75,845.85,1088.75,2686.05,84.90,856.55,1258.45,263.00,327.50,433.25,963.85,1105.25,1027.00,727.25,1289.20,1265.35,144.50,242.50,141.00,1015.35,227.80,815.55,2416.40,401.90,257.40,520.05,352.05,440.35,2824.00,556.45
1,257.95,880.80,438.40,2483.35,6052.45,891.45,326.80,2972.60,645.65,1149.70,3056.00,359.80,3657.25,845.95,1070.50,2637.80,80.75,859.00,1216.70,255.55,325.10,426.45,934.20,1078.90,1044.70,705.30,1255.95,1242.50,143.30,238.05,140.95,995.30,220.70,799.10,2369.60,377.05,256.90,516.40,351.40,433.85,2749.15,557.70
2,255.70,900.65,436.45,2502.35,5997.85,902.65,323.45,2966.70,640.85,1147.50,3070.65,372.25,3659.10,842.80,1062.40,2616.85,82.60,847.95,1209.40,256.70,324.85,442.65,935.25,1074.05,1083.10,707.75,1256.90,1249.20,142.25,241.85,139.15,1005.15,217.75,800.50,2348.95,374.45,274.30,521.90,347.70,435.30,2733.05,556.90
3,251.80,886.40,430.70,2485.50,6107.25,912.30,322.20,3001.70,652.10,1137.20,3046.25,372.70,3652.40,841.40,1067.10,2578.20,80.55,843.05,1209.30,250.10,315.10,450.85,943.95,1069.35,1058.20,703.60,1236.85,1226.15,141.45,237.55,139.50,1032.20,216.85,791.95,2381.60,365.90,268.75,525.10,347.85,443.35,2735.30,555.10
4,245.00,872.35,409.25,2419.25,6106.30,884.25,322.35,2946.55,637.45,1126.95,2991.80,370.60,3557.20,825.05,1056.20,2520.10,76.65,820.25,1179.45,246.75,309.60,441.55,925.50,1050.80,1026.35,691.65,1206.40,1197.45,137.85,226.60,138.10,1013.25,209.55,783.80,2371.25,343.55,249.90,510.80,344.05,417.40,2667.70,549.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,499.45,2793.85,664.45,3492.65,5119.00,392.00,514.00,3551.10,827.25,3842.10,5286.90,129.30,961.30,991.35,1426.70,3067.20,250.30,2450.55,2651.85,537.25,211.45,93.00,921.65,1293.80,395.25,1959.75,1306.30,740.10,97.60,94.95,188.00,1966.10,281.75,603.45,3093.00,193.20,680.55,1003.85,1570.95,471.25,5341.20,406.30
1241,496.80,2805.35,654.25,3462.70,5030.30,387.30,525.30,3539.70,824.80,3879.85,5288.30,134.00,984.60,978.20,1420.55,3083.55,259.05,2417.30,2638.85,546.70,205.40,94.10,922.35,1282.10,401.70,1970.40,1314.00,736.10,97.85,96.95,196.15,1914.25,285.05,605.30,3051.50,195.40,683.80,997.15,1572.60,472.40,5448.35,406.40
1242,513.85,2792.25,671.10,3437.95,5081.00,392.20,545.25,3552.80,826.55,3803.05,5270.90,133.45,993.85,962.55,1416.25,3055.25,272.90,2368.85,2661.35,541.10,202.80,94.50,952.05,1262.15,405.40,1952.40,1338.95,744.40,97.00,97.90,197.05,1911.15,287.70,601.90,3032.80,196.75,722.80,994.75,1542.35,482.50,5397.95,406.75
1243,517.00,2844.70,672.70,3529.15,5082.00,400.15,540.25,3575.25,838.70,3859.15,5338.25,132.75,1004.30,994.65,1431.65,3161.10,268.20,2391.20,2657.50,542.05,201.50,96.15,939.80,1312.10,402.85,1970.70,1373.40,770.50,100.15,100.65,203.55,1933.70,286.00,620.80,3120.90,198.15,713.15,1051.10,1548.60,503.65,5591.75,430.20


In [ ]:
stonks=[]
for i in x:
  stonks.append(i)
len(stonks)

42

In [ ]:
alldata=x   #the original dataset

In [ ]:
timesteps=45                                     #lstm hyperparameters "Subject to be tuned"
epoch=50
batchSize=32
neurons=50
after2020=len(y.loc["2021-01-01" : ])                    #number of days after 31-12-2020 "automated"

In [ ]:
def create_dataset(dataset, time_step=1):         # convert an array of values into a dataset matrix which will be used to train the lstm model.
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step):
		a = dataset[i:(i+time_step), 0]               #i=0, 0,1,2,3-----(timesteps-1)  -> timesteps
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
def forcast(df1, timesteps, epoch, batchSize, neurons, after2020):             
  scaler=MinMaxScaler(feature_range=(0,1))   
  df1=scaler.fit_transform(np.array(df1).reshape(-1,1))           #minmax scalar transformation of data

  before_2021_data_length=int(len(df1)-after2020)                 #length of data before 2021
  training_size=int(before_2021_data_length*0.80)                 #80% of training size, refered from Yadav et al (2020) (Science Direct)
  train_data=df1[0:training_size,:]                              
  test_data=df1[training_size:before_2021_data_length,:1]         #20% of testing data, refered from Yadav et al (2020) (Science Direct)                
  inpdata=df1[before_2021_data_length-timesteps:len(df1),:1]      #getting the data from 01-01-2021 onwards


  #reshape into X=t,t+1,t+2,t+3,........t+"timestep-1" and Y=t+"timestep"
  X_train, y_train = create_dataset(train_data, timesteps)
  x_inp, y_inp = create_dataset(inpdata, timesteps)
  x_test, y_test = create_dataset(test_data,timesteps)

  X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1) 
  x_inp = x_inp.reshape(x_inp.shape[0],x_inp.shape[1] , 1)        #reshape input to be [samples, time steps, features] which is required for LSTM
  x_test = x_test.reshape(x_test.shape[0],x_test.shape[1] , 1)

  # initialising stacked lstm
  model=Sequential()
  model.add(LSTM(neurons,return_sequences=True,input_shape=(timesteps,1)))
  model.add(LSTM(neurons,return_sequences=True))
  model.add(LSTM(neurons))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error',optimizer='adam')

  model.fit(X_train,y_train,validation_data=(x_test,y_test),epochs=epoch,batch_size=batchSize,verbose=1)     # training of the model

  test_predict=model.predict(x_test)                    #prediction using test data as input

  #performance metrics between, original test data and predicted test data
  msetst =mean_squared_error(y_test,test_predict)
  rmsetst=math.sqrt(msetst)
  maetst =mean_absolute_error(y_test,test_predict)
  r2tst  =r2_score(y_test,test_predict)
  mapetst=mean_absolute_percentage_error(y_test,test_predict)
  tstlst =[msetst,rmsetst,maetst,r2tst,mapetst]


  #model is trained again on the test data so as to increase the learning (it is often termed as incremental learning)
  #refered from url: https://www.justintodata.com/forecast-time-series-lstm-with-tensorflow-keras/#step-2-transforming-the-dataset-for-tensorflow-keras
  #refered from url: https://github.com/keras-team/keras/issues/4446
  model.fit(x_test,y_test,epochs=epoch,batch_size=batchSize,verbose=1)
  
  out_predict=model.predict(x_inp)                      #dynamic prediction of the stock's closing price from 01-01-2021 onwards

  #performance metrics between, original data(after 31-12-2020) and dynamically predicted data (after 31-12-2020)
  mseinp =mean_squared_error(y_inp,out_predict)
  rmseinp=math.sqrt(mseinp)
  maeinp =mean_absolute_error(y_inp,out_predict)
  r2inp  =r2_score(y_inp,out_predict)
  mapeinp=mean_absolute_percentage_error(y_inp,out_predict)
  inplst =[mseinp,rmseinp,maeinp,r2inp,mapeinp]


  lst=[]
  for i in out_predict:
    lst.append(i)

  p=train_data.tolist()
  q=test_data.tolist()
  p.extend(q)                                         #appending train and test data to make dataset before 2021 (data till 31-12-2020)
  p.extend(lst)                                       #appending the data, forcasted from 01-01-2021 onwards, to the data till 31-12-2020
  p=scaler.inverse_transform(p).tolist()

  return pd.DataFrame(p), tstlst, inplst
  #returns a dataframe, tstlst => test performance metrics, inplst => forcasted data performance metrics
    

In [ ]:
mtest=[]
mdynamic=[]
fdata=pd.DataFrame()
for i in alldata:                                   # this for loop will be iterated for 42 times i.e. for each column of the original dataset
  temp=alldata[i]
  ftemp,trmse,drmse=forcast(temp, timesteps, epoch, batchSize, neurons, after2020)    #hyperparameters are provided as input here
  fdata = pd.concat([fdata,ftemp],axis = 1)
  mtest.append(trmse)
  mdynamic.append(drmse)
fdata.columns=stonks

Epoch 1/50
30/30 [==============================] - 9s 119ms/step - loss: 0.1082 - val_loss: 0.0125
Epoch 2/50
30/30 [==============================] - 2s 71ms/step - loss: 0.0053 - val_loss: 0.0101
Epoch 3/50
30/30 [==============================] - 2s 73ms/step - loss: 0.0031 - val_loss: 0.0093
Epoch 4/50
30/30 [==============================] - 2s 71ms/step - loss: 0.0027 - val_loss: 0.0091
Epoch 5/50
30/30 [==============================] - 2s 73ms/step - loss: 0.0025 - val_loss: 0.0089
Epoch 6/50
30/30 [==============================] - 2s 74ms/step - loss: 0.0027 - val_loss: 0.0082
Epoch 7/50
30/30 [==============================] - 2s 75ms/step - loss: 0.0025 - val_loss: 0.0078
Epoch 8/50
30/30 [==============================] - 2s 72ms/step - loss: 0.0025 - val_loss: 0.0074
Epoch 9/50
30/30 [==============================] - 2s 73ms/step - loss: 0.0023 - val_loss: 0.0072
Epoch 10/50
30/30 [==============================] - 2s 73ms/step - loss: 0.0026 - val_loss: 0.0066
Epoch 11

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


7/7 [==============================] - 0s 70ms/step - loss: 0.0026
Epoch 2/50
7/7 [==============================] - 0s 70ms/step - loss: 0.0021
Epoch 3/50
7/7 [==============================] - 1s 73ms/step - loss: 0.0021
Epoch 4/50
7/7 [==============================] - 0s 68ms/step - loss: 0.0020
Epoch 5/50
7/7 [==============================] - 0s 67ms/step - loss: 0.0019
Epoch 6/50
7/7 [==============================] - 0s 69ms/step - loss: 0.0017
Epoch 7/50
7/7 [==============================] - 0s 72ms/step - loss: 0.0017
Epoch 8/50
7/7 [==============================] - 1s 71ms/step - loss: 0.0019
Epoch 9/50
7/7 [==============================] - 0s 70ms/step - loss: 0.0017
Epoch 10/50
7/7 [==============================] - 1s 70ms/step - loss: 0.0018
Epoch 11/50
7/7 [==============================] - 1s 71ms/step - loss: 0.0019
Epoch 12/50
7/7 [==============================] - 1s 71ms/step - loss: 0.0019
Epoch 13/50
7/7 [==============================] - 0s 70ms/step - loss: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


7/7 [==============================] - 0s 67ms/step - loss: 0.0020
Epoch 2/50
7/7 [==============================] - 0s 65ms/step - loss: 0.0017
Epoch 3/50
7/7 [==============================] - 0s 64ms/step - loss: 0.0019
Epoch 4/50
7/7 [==============================] - 0s 66ms/step - loss: 0.0016
Epoch 5/50
7/7 [==============================] - 0s 64ms/step - loss: 0.0018
Epoch 6/50
7/7 [==============================] - 0s 68ms/step - loss: 0.0020
Epoch 7/50
7/7 [==============================] - 0s 66ms/step - loss: 0.0019
Epoch 8/50
7/7 [==============================] - 0s 70ms/step - loss: 0.0015
Epoch 9/50
7/7 [==============================] - 0s 65ms/step - loss: 0.0015
Epoch 10/50
7/7 [==============================] - 0s 67ms/step - loss: 0.0016
Epoch 11/50
7/7 [==============================] - 0s 67ms/step - loss: 0.0018
Epoch 12/50
7/7 [==============================] - 0s 66ms/step - loss: 0.0027
Epoch 13/50
7/7 [==============================] - 0s 67ms/step - loss: 

...

7/7 [==============================] - 0s 71ms/step - loss: 3.4405e-04
Epoch 2/50
7/7 [==============================] - 1s 69ms/step - loss: 3.3176e-04
Epoch 3/50
7/7 [==============================] - 1s 73ms/step - loss: 3.2650e-04
Epoch 4/50
7/7 [==============================] - 0s 68ms/step - loss: 3.2344e-04
Epoch 5/50
7/7 [==============================] - 1s 74ms/step - loss: 3.1953e-04
Epoch 6/50
7/7 [==============================] - 1s 71ms/step - loss: 3.2824e-04
Epoch 7/50
7/7 [==============================] - 1s 72ms/step - loss: 2.9066e-04
Epoch 8/50
7/7 [==============================] - 1s 70ms/step - loss: 2.8800e-04
Epoch 9/50
7/7 [==============================] - 1s 69ms/step - loss: 2.8229e-04
Epoch 10/50
7/7 [==============================] - 1s 69ms/step - loss: 2.8458e-04
Epoch 11/50
7/7 [==============================] - 1s 75ms/step - loss: 2.7425e-04
Epoch 12/50
7/7 [==============================] - 0s 67ms/step - loss: 2.6951e-04
Epoch 13/50
7/7 [=======

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


7/7 [==============================] - 1s 74ms/step - loss: 0.0015
Epoch 2/50
7/7 [==============================] - 0s 68ms/step - loss: 0.0014
Epoch 3/50
7/7 [==============================] - 1s 70ms/step - loss: 0.0011
Epoch 4/50
7/7 [==============================] - 1s 69ms/step - loss: 9.4922e-04
Epoch 5/50
7/7 [==============================] - 1s 70ms/step - loss: 8.3685e-04
Epoch 6/50
7/7 [==============================] - 0s 70ms/step - loss: 8.7353e-04
Epoch 7/50
7/7 [==============================] - 1s 70ms/step - loss: 7.8656e-04
Epoch 8/50
7/7 [==============================] - 1s 71ms/step - loss: 7.5770e-04
Epoch 9/50
7/7 [==============================] - 1s 72ms/step - loss: 7.4052e-04
Epoch 10/50
7/7 [==============================] - 0s 67ms/step - loss: 7.6294e-04
Epoch 11/50
7/7 [==============================] - 1s 73ms/step - loss: 7.2895e-04
Epoch 12/50
7/7 [==============================] - 1s 71ms/step - loss: 7.3752e-04
Epoch 13/50
7/7 [===================

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


Epoch 1/50
7/7 [==============================] - 0s 68ms/step - loss: 4.7230e-04
Epoch 2/50
7/7 [==============================] - 0s 70ms/step - loss: 4.5900e-04
Epoch 3/50
7/7 [==============================] - 0s 70ms/step - loss: 4.8896e-04
Epoch 4/50
7/7 [==============================] - 0s 71ms/step - loss: 4.4658e-04
Epoch 5/50
7/7 [==============================] - 1s 71ms/step - loss: 4.4690e-04
Epoch 6/50
7/7 [==============================] - 1s 72ms/step - loss: 4.5034e-04
Epoch 7/50
7/7 [==============================] - 1s 75ms/step - loss: 4.4013e-04
Epoch 8/50
7/7 [==============================] - 1s 72ms/step - loss: 4.4956e-04
Epoch 9/50
7/7 [==============================] - 1s 73ms/step - loss: 4.5891e-04
Epoch 10/50
7/7 [==============================] - 0s 69ms/step - loss: 4.4482e-04
Epoch 11/50
7/7 [==============================] - 1s 69ms/step - loss: 4.1774e-04
Epoch 12/50
7/7 [==============================] - 0s 70ms/step - loss: 4.1277e-04
Epoch 13/50
7

In [ ]:
fdata # dataset with 2021 rows forcasted dynamically

,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BPCL,BHARTIARTL,BRITANNIA,CIPLA,DIVISLAB,DRREDDY,GAIL,GRASIM,HCLTECH,HDFCBANK,HEROMOTOCO,HINDALCO,HINDUNILVR,HDFC,ICICIBANK,ITC,IOC,INDUSINDBK,INFY,JSWSTEEL,KOTAKBANK,LT,M&M,NTPC,ONGC,POWERGRID,RELIANCE,SBIN,SUNPHARMA,TCS,TATAMOTORS,TATASTEEL,TECHM,TITAN,UPL,ULTRACEMCO,WIPRO
0,267.550000,878.750000,449.900000,2518.000000,6095.850000,896.050000,340.500000,2986.400000,655.350000,1162.500000,3108.600000,369.750000,3767.750000,845.850000,1088.750000,2686.050000,84.900000,856.550000,1258.450000,263.000000,327.500000,433.250000,963.850000,1105.250000,1027.000000,727.250000,1289.200000,1265.350000,144.500000,242.500000,141.000000,1015.350000,227.800000,815.550000,2416.400000,401.900000,257.400000,520.050000,352.050000,440.350000,2824.000000,556.450000
1,257.950000,880.800000,438.400000,2483.350000,6052.450000,891.450000,326.800000,2972.600000,645.650000,1149.700000,3056.000000,359.800000,3657.250000,845.950000,1070.500000,2637.800000,80.750000,859.000000,1216.700000,255.550000,325.100000,426.450000,934.200000,1078.900000,1044.700000,705.300000,1255.950000,1242.500000,143.300000,238.050000,140.950000,995.300000,220.700000,799.100000,2369.600000,377.050000,256.900000,516.400000,351.400000,433.850000,2749.150000,557.700000
2,255.700000,900.650000,436.450000,2502.350000,5997.850000,902.650000,323.450000,2966.700000,640.850000,1147.500000,3070.650000,372.250000,3659.100000,842.800000,1062.400000,2616.850000,82.600000,847.950000,1209.400000,256.700000,324.850000,442.650000,935.250000,1074.050000,1083.100000,707.750000,1256.900000,1249.200000,142.250000,241.850000,139.150000,1005.150000,217.750000,800.500000,2348.950000,374.450000,274.300000,521.900000,347.700000,435.300000,2733.050000,556.900000
3,251.800000,886.400000,430.700000,2485.500000,6107.250000,912.300000,322.200000,3001.700000,652.100000,1137.200000,3046.250000,372.700000,3652.400000,841.400000,1067.100000,2578.200000,80.550000,843.050000,1209.300000,250.100000,315.100000,450.850000,943.950000,1069.350000,1058.200000,703.600000,1236.850000,1226.150000,141.450000,237.550000,139.500000,1032.200000,216.850000,791.950000,2381.600000,365.900000,268.750000,525.100000,347.850000,443.350000,2735.300000,555.100000
4,245.000000,872.350000,409.250000,2419.250000,6106.300000,884.250000,322.350000,2946.550000,637.450000,1126.950000,2991.800000,370.600000,3557.200000,825.050000,1056.200000,2520.100000,76.650000,820.250000,1179.450000,246.750000,309.600000,441.550000,925.500000,1050.800000,1026.350000,691.650000,1206.400000,1197.450000,137.850000,226.600000,138.100000,1013.250000,209.550000,783.800000,2371.250000,343.550000,249.900000,510.800000,344.050000,417.400000,2667.700000,549.850000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,498.497959,2741.156619,627.238341,3510.294288,5221.962159,386.294796,528.734187,3540.325745,848.654303,3832.095812,5232.518460,118.224148,931.434410,979.998197,1396.007967,3076.302690,248.105744,2406.034391,2567.593628,536.388217,212.659247,89.480008,923.142898,1294.905735,393.908807,1964.297013,1308.573164,727.442855,96.854800,88.563335,186.525644,1990.201849,277.551965,604.436410,2962.955125,182.757519,677.566453,985.959115,1566.066333,465.322576,5355.102489,401.433570
1241,496.804314,2778.718042,658.057181,3483.381524,5129.978827,389.637727,526.147702,3531.301705,845.339618,3825.229236,5241.211679,119.586891,941.151461,997.129057,1392.639482,3061.394587,251.366043,2427.946142,2634.341780,542.927315,211.933584,89.989873,927.742005,1298.218539,397.554362,1954.165434,1305.556305,735.751768,96.735943,89.212142,185.753041,1964.055366,278.457578,605.717679,3072.559980,186.364017,685.981483,995.613706,1576.751103,466.770558,5392.461277,413.310767
1242,494.156159,2796.518212,665.568268,3466.151039,5011.232562,389.793720,535.182744,3522.502761,843.612987,3838.273306,5239.944557,121.622286

In [ ]:
alldata # dataset with original 2021 rows

,ADANIPORTS,ASIANPAINT,AXISBANK,BAJAJ-AUTO,BAJFINANCE,BPCL,BHARTIARTL,BRITANNIA,CIPLA,DIVISLAB,DRREDDY,GAIL,GRASIM,HCLTECH,HDFCBANK,HEROMOTOCO,HINDALCO,HINDUNILVR,HDFC,ICICIBANK,ITC,IOC,INDUSINDBK,INFY,JSWSTEEL,KOTAKBANK,LT,M&M,NTPC,ONGC,POWERGRID,RELIANCE,SBIN,SUNPHARMA,TCS,TATAMOTORS,TATASTEEL,TECHM,TITAN,UPL,ULTRACEMCO,WIPRO
0,267.55,878.75,449.90,2518.00,6095.85,896.05,340.50,2986.40,655.35,1162.50,3108.60,369.75,3767.75,845.85,1088.75,2686.05,84.90,856.55,1258.45,263.00,327.50,433.25,963.85,1105.25,1027.00,727.25,1289.20,1265.35,144.50,242.50,141.00,1015.35,227.80,815.55,2416.40,401.90,257.40,520.05,352.05,440.35,2824.00,556.45
1,257.95,880.80,438.40,2483.35,6052.45,891.45,326.80,2972.60,645.65,1149.70,3056.00,359.80,3657.25,845.95,1070.50,2637.80,80.75,859.00,1216.70,255.55,325.10,426.45,934.20,1078.90,1044.70,705.30,1255.95,1242.50,143.30,238.05,140.95,995.30,220.70,799.10,2369.60,377.05,256.90,516.40,351.40,433.85,2749.15,557.70
2,255.70,900.65,436.45,2502.35,5997.85,902.65,323.45,2966.70,640.85,1147.50,3070.65,372.25,3659.10,842.80,1062.40,2616.85,82.60,847.95,1209.40,256.70,324.85,442.65,935.25,1074.05,1083.10,707.75,1256.90,1249.20,142.25,241.85,139.15,1005.15,217.75,800.50,2348.95,374.45,274.30,521.90,347.70,435.30,2733.05,556.90
3,251.80,886.40,430.70,2485.50,6107.25,912.30,322.20,3001.70,652.10,1137.20,3046.25,372.70,3652.40,841.40,1067.10,2578.20,80.55,843.05,1209.30,250.10,315.10,450.85,943.95,1069.35,1058.20,703.60,1236.85,1226.15,141.45,237.55,139.50,1032.20,216.85,791.95,2381.60,365.90,268.75,525.10,347.85,443.35,2735.30,555.10
4,245.00,872.35,409.25,2419.25,6106.30,884.25,322.35,2946.55,637.45,1126.95,2991.80,370.60,3557.20,825.05,1056.20,2520.10,76.65,820.25,1179.45,246.75,309.60,441.55,925.50,1050.80,1026.35,691.65,1206.40,1197.45,137.85,226.60,138.10,1013.25,209.55,783.80,2371.25,343.55,249.90,510.80,344.05,417.40,2667.70,549.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240,499.45,2793.85,664.45,3492.65,5119.00,392.00,514.00,3551.10,827.25,3842.10,5286.90,129.30,961.30,991.35,1426.70,3067.20,250.30,2450.55,2651.85,537.25,211.45,93.00,921.65,1293.80,395.25,1959.75,1306.30,740.10,97.60,94.95,188.00,1966.10,281.75,603.45,3093.00,193.20,680.55,1003.85,1570.95,471.25,5341.20,406.30
1241,496.80,2805.35,654.25,3462.70,5030.30,387.30,525.30,3539.70,824.80,3879.85,5288.30,134.00,984.60,978.20,1420.55,3083.55,259.05,2417.30,2638.85,546.70,205.40,94.10,922.35,1282.10,401.70,1970.40,1314.00,736.10,97.85,96.95,196.15,1914.25,285.05,605.30,3051.50,195.40,683.80,997.15,1572.60,472.40,5448.35,406.40
1242,513.85,2792.25,671.10,3437.95,5081.00,392.20,545.25,3552.80,826.55,3803.05,5270.90,133.45,993.85,962.55,1416.25,3055.25,272.90,2368.85,2661.35,541.10,202.80,94.50,952.05,1262.15,405.40,1952.40,1338.95,744.40,97.00,97.90,197.05,1911.15,287.70,601.90,3032.80,196.75,722.80,994.75,1542.35,482.50,5397.95,406.75
1243,517.00,2844.70,672.70,3529.15,5082.00,400.15,540.25,3575.25,838.70,3859.15,5338.25,132.75,1004.30,994.65,1431.65,3161.10,268.20,2391.20,2657.50,542.05,201.50,96.15,939.80,1312.10,402.85,1970.70,1373.40,770.50,100.15,100.65,203.55,1933.70,286.00,620.80,3120.90,198.15,713.15,1051.10,1548.60,503.65,5591.75,430.20


In [ ]:
fdata.to_csv('/content/drive/My Drive/experiment/take 14-03-2021/data_inc/fdata.csv')   #dataset saved in .csv format

In [ ]:
clm=['MSE','RMSE','MAE','R2','MAPE']
pd.DataFrame(mtest,index=stonks,columns=clm).to_csv('/content/drive/My Drive/experiment/take 14-03-2021/data_inc/mtest.csv') #metric values saved 

In [ ]:
pd.DataFrame(mdynamic,index=stonks,columns=clm).to_csv('/content/drive/My Drive/experiment/take 14-03-2021/data_inc/mdynamic.csv') #metric values saved 